In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import os

#change path
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'python_lesson/CloneCoding/펭귄 몸무게 예측 경진대회'
os.chdir(base_dir)

#mount google driver
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#root_dir = "/content/gdrive/My Drive/"

!pwd

/content/drive/My Drive/python_lesson/CloneCoding/펭귄 몸무게 예측 경진대회


# 펭귄 몸무게 예측 경진대회 - K겹 교차검증을 위한 모델의 과적합 방지

---

In [32]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

# csv 형식으로 된 데이터 파일을 읽어옵니다.
data = pd.read_csv('data/penguins.csv')

# 데이터의 최상단 5 줄을 표시합니다.
data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181,3750,Male
1,Adelie,Torgersen,39.5,17.4,186,3800,Female
2,Adelie,Torgersen,40.3,18.0,195,3250,Female
3,Adelie,Torgersen,36.7,19.3,193,3450,Female
4,Adelie,Torgersen,39.3,20.6,190,3650,Male


### 인코딩

결측치가 존재하지 않은건 이전에 확인했으니 넘어가겠습니다. <br>

이번에는 범주형 변수들을 숫자 형태로 인코딩 해주겠습니다.

#### 라벨 인코딩

라벨 인코딩이란, 범주형 변수를 숫자로 인코딩해서 동시에 컴퓨터가 이들이 각각 다른 범주인 것을 인식하도록 값을 0,1,2 등의 값으로 구별하여 인코딩하는 기법을 말합니다.

In [33]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object': 
            label_map = {'unknown':0} 
            for i, key in enumerate(data[col].unique()):
                label_map[key] = i+1
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값으로 채워줍니다.
    return dataframe

# 전처리 완료한 data
label_map = make_label_map(data) # data 사용해 label map 생성

labeled_data = label_encoder(data, label_map) # data 라벨 인코딩

In [34]:
label_map

{'island': {'Biscoe': 2, 'Dream': 3, 'Torgersen': 1, 'unknown': 0},
 'sex': {'Female': 2, 'Male': 1, 'unknown': 0},
 'species': {'Adelie': 1, 'Chinstrap': 3, 'Gentoo': 2, 'unknown': 0}}

### 데이터 분할

In [35]:
# train/test로 분할

from sklearn.model_selection import train_test_split

train, test = train_test_split(labeled_data, test_size=0.2, shuffle=True)

print(train.shape)
print(test.shape)

(266, 7)
(67, 7)


### 모델링

산식은 마찬가지로 대회에 정해진 RMSE를 사용합니다.

In [36]:
# 대회 규칙의 평가 산식 함수를 그대로 사용합니다.
import numpy as np

def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

### K-Fold Cross Validation 사용하기

1. 전체 데이터셋을 Training Set과 Test Set으로 나눕니다.
2. Training Set를 k개의 폴드(부분)로 나눕니다.
3. 첫 번째 폴드를 Validation Set으로 사용하고 나머지 폴드들을 Training Set으로 사용합니다.
4. 모델을 Training한 뒤, 첫 번 째 Validation Set으로 평가합니다.
5. 차례대로 다음 폴드를 Validation Set으로 사용하며 K번을 반복합니다.

결과적으로 총 k 개의 성능 결과를 얻게 되면, 모델의 성능은 각 결과값들의 평균으로 정해집니다.

하단 이미지는 5개의 fold를 사용한 Cross Validation 과정을 보여줍니다.

<img width=600 src=https://user-images.githubusercontent.com/48666867/148737283-369a638f-2bb9-4115-8df7-8ee5ac2188c2.png>

하나의 데이터셋을 가지고 모든 데이터를 적어도 한번은 학습에 활용하며 K번의 검증 과정을 거칠 수 있습니다.

이를 통해 조금 더 일반화된 모델을 만들 수 있습니다.

In [37]:
from sklearn.linear_model import LinearRegression # 선형 회귀 모델을 불러옵니다.
from sklearn.model_selection import KFold # 데이터를 K개의 폴드로 분할해주는 패키지를 사용하겠습니다.

In [38]:
target = train["body_mass_g"] # 타겟 변수인 "body_mass_g"
print(target.shape)

feature = train.drop(['body_mass_g'], axis=1) # 학습에 불필요한 타겟 변수인 "Body Mass (g)"을 제외한 모든 features
print(feature.shape)

(266,)
(266, 6)


In [39]:
# 예측모델 인스턴스를 만듭니다.
lr = LinearRegression()

# KFold 함수를 이용해 폴드 개수를 할당해줍니다.(임의로 5개의 폴드를 사용합니다. 일반적으로 5,10 등의 값을 사용합니다.)
kfold = KFold(n_splits=5) 

In [40]:
cv_rmse = [] # 각 cv회차의 rmse 점수를 계산하여 넣어줄 리스트를 생성합니다. 이후 RMSE값의 평균을 구하기 위해 사용됩니다.
n_iter =0 # 반복 횟수 값을 초기 설정해줍니다. 이후 프린트문에서 각 교차검증의 회차를 구분하기 위해 사용됩니다.

# K값이 5이므로 이 반복문은 5번 반복하게 됩니다.
for train_index, test_index in kfold.split(feature):  # feautres 데이터를 위에서 지정한 kfold 숫자로 분할합니다. 인덱스 값을 분할해줍니다.

    x_train, x_test = feature.iloc[train_index], feature.iloc[test_index] # feature로 사용할 값을 나눠진 인덱스값에 따라 설정합니다.
    y_train, y_test = target.iloc[train_index], target.iloc[test_index] # label로 사용할 값을 나눠진 인덱스값에 따라 설정합니다.
    
    lr = lr.fit(x_train, y_train) # 모델 학습
    pred = lr.predict(x_test) # 테스트셋 예측
    n_iter += 1 # 반복 횟수 1회 증가
    
    error = RMSE(y_test, pred) # RMSE 점수를 구합니다.
    train_size = x_train.shape[0] # 학습 데이터 크기
    test_size = x_test.shape[0] # 검증 데이터 크기
    
    print('\n{0}번째 교차 검증 RMSE : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, error, train_size, test_size))
    print('{0}번째 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_rmse.append(error)
    
print('\n==> 이 방정식의 평균 에러(RMSE)는 {} 입니다.'.format(np.mean(cv_rmse))) # 모델의 평균정확도를 확인합니다.


1번째 교차 검증 RMSE : 368.6019407339491,  학습 데이터 크기 : 212,  검증 데이터 크기 : 54
1번째 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53]

2번째 교차 검증 RMSE : 386.5862471360172,  학습 데이터 크기 : 213,  검증 데이터 크기 : 53
2번째 검증 세트 인덱스 : [ 54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106]

3번째 교차 검증 RMSE : 347.21265819475417,  학습 데이터 크기 : 213,  검증 데이터 크기 : 53
3번째 검증 세트 인덱스 : [107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159]

4번째 교차 검증 RMSE : 287.71986746163225,  학습 데이터 크기 : 213,  검증 데이터 크기 : 53
4번째 검증 세트 인덱스 : [160 161 162 163 164 165 166 167 168 169 170 171 172

### test 예측하기

In [41]:
test

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,1,1,39.1,18.7,181,3750,1
330,3,3,49.6,18.2,193,3775,1
64,1,1,33.5,19.0,190,3600,2
162,2,2,42.0,13.5,210,4150,2
245,2,2,55.9,17.0,228,5600,1
...,...,...,...,...,...,...,...
259,2,2,48.8,16.2,222,6000,1
139,1,3,39.0,18.7,185,3650,1
69,1,1,42.8,18.5,195,4250,1
4,1,1,39.3,20.6,190,3650,1


In [42]:
# test 데이터에 Y값이 없다고 가정하고 실제로 주어지지 않은 데이터를 예측해보겠습니다.
test = test.drop(['body_mass_g'], axis=1)
test.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,sex
0,1,1,39.1,18.7,181,1
330,3,3,49.6,18.2,193,1
64,1,1,33.5,19.0,190,2
162,2,2,42.0,13.5,210,2
245,2,2,55.9,17.0,228,1


In [43]:
predict_test = lr.predict(test)

predict_test[:5]

array([3645.44769303, 4004.34531864, 3308.76910206, 4471.03365547,
       5762.14487565])

## Reference

1. https://www.dacon.io/competitions/official/235862/codeshare/4071?page=1&dtype=recent
2. https://www.kaggle.com/resulcaliskan/penguins